In [1]:
import googlemaps
import requests

In [2]:
API_KEY = open("./API_KEY.txt").read()
map_client = googlemaps.Client(API_KEY)

In [5]:
my_location = map_client.geolocate()

{'location': {'lat': 29.7482228, 'lng': 31.2511796},
 'accuracy': 1574.3131999248544}

In [4]:
colleg_loc = (30.0263349,31.2094813) # الكلية
helwan_metro_loc = (29.8489866,31.3320422) # مترو حلوان

In [ ]:
# TESTING !!!
# def search_by_text(text, location=None, is_next= False):
#     url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query="+ text +"&key=" + API_KEY

#     payload={}
#     headers = {}

#     response = requests.request("GET", url, headers=headers, data=payload)
    
#     json_res = response.json()
#     query = json_res["results"]

#     return query


# search_by_text("عايز مطاعم", my_location)

In [6]:
'''
Useful params in places_nearby
    max_price [0 , 4] "4 is most expensive"
    type
    radius
'''
res = map_client.places_nearby(location=(29.8489139,31.3360409), radius=1000, type="clothing_store")

In [10]:
'''
distance API:
    destination_addresses
    distance in km
    duration
'''
# map_client.distance_matrix((29.8131478,31.3031655), helwan_metro_loc)

'\ndistance API:\n    destination_addresses\n    distance in km\n    duration\n'

In [11]:
'''
name
opening_hours : {'open_now': True}
vicinity
rating
user_ratings_total
geometry
price_level
'''

"\nname\nopening_hours : {'open_now': True}\nvicinity\nrating\nuser_ratings_total\ngeometry\nprice_level\n"

In [12]:
# for x in res["results"]:
#     if "restaurant" in x["types"]:
#         print("Restaurant: ", x["name"])
#     elif "hospital" in x["types"]:
#         print("Hospital: ", x["name"])
#     elif "lodging" in x["types"]:
#         print("Lodging: ", x["name"])
#     elif "university" in x["types"]:
#         print("University: ", x["name"])
#     elif "travel_agency" in x["types"]:
#         print("Travel agency: ", x["name"])
#     elif "doctor" in x["types"]:
#         print("Doctor: ", x["name"])
#     elif "clothing_store" in x["types"]:
#         print("Clothes: ", x["name"])


In [28]:
class location_recommender:
    def __init__(self) -> None:
        self.__API_KEY = open("./API_KEY.txt").read()
        self.map_client = googlemaps.Client(self.__API_KEY)
        self.requests = requests
        pass

    def __formating_results(self, query_results, my_location= None):

        results = []

        for res in query_results:
            new_location = (res["geometry"]["location"]["lat"], res["geometry"]["location"]["lng"])
            new_entry = {}

            if my_location != None: new_entry["distance"] = self.map_client.distance_matrix(my_location, new_location)

            if "name" in res.keys(): new_entry["name"] = res["name"]
            if "types" in res.keys(): new_entry["types"] = res["types"]
            if "vicinity" in res.keys(): new_entry["vicinity"] = res["vicinity"]
            if "geometry" in res.keys(): new_entry["geometry"] = res["geometry"]
            if "business_status" in res.keys(): new_entry["business_status"] = res["business_status"]
            if "price_level" in res.keys(): new_entry["price_level"] = res["price_level"]
            if "rating" in res.keys(): new_entry["rating"] = res["rating"]
            if "user_ratings_total" in res.keys(): new_entry["user_ratings_total"] = res["user_ratings_total"]
            if "formatted_address" in res.keys(): new_entry["formatted_address"] = res["formatted_address"]

            results.append(new_entry)

        return results


    def recommend(self, my_location, categ=None, keyword=None, radius=1000, is_rankby_distange= False):
        # @categ => [restaurant, clothing_store, hospital, doctor, ...]

        results = []
        query = None

        if categ == None and keyword == None:
            return []

        try:
            if is_rankby_distange == False:
                query = self.map_client.places_nearby(my_location, keyword=keyword, type=categ, radius=radius)
            else:
                query = self.map_client.places_nearby(my_location, keyword=keyword, type=categ, rank_by="distance")

            query = query["results"]

            results = self.__formating_results(query, my_location)
            return results

        except:
            return []

    def search_by_text(self, text, is_next= False):
        url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query="+ text +"&key=" + self.__API_KEY

        payload={}
        headers = {}

        response = self.requests.request("GET", url, headers=headers, data=payload)
        
        json_res = response.json()
        query = json_res["results"]

        results = self.__formating_results(query)
        return results
    
    def get_my_location(self):
        location = self.map_client.geolocate()
        return location["location"]

In [29]:
model = location_recommender()
# res = model.recommend(colleg_loc, "restaurant", 1000)

In [32]:
# model.recommend(colleg_loc, "restaurant")

In [ ]:
# model.recommend(helwan_metro_loc, keyword="مطاعم بيتزا")

In [12]:
# tmp = model.search_by_text("مطعم في حلوان")

In [16]:
import dill as pickle

path = "../utils/location_recommender"

with open(path ,'wb') as f:
    pickle.dump(model, f)